#### spateGAN-ERA5 spatiotemporal precipitation downscaling example
https://doi.org/10.48550/arXiv.2411.16098

In [1]:
%load_ext autoreload
%autoreload 2
import torch
from model import Generator  
from train import train, validation
from data import readFile, make_temporal_batches, block_average_pooling
import xarray as xr
import numpy as np
from einops import rearrange

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


#### load model and weights and run the model


#### AORC to AORC

In [ ]:
data_aorc = readFile('data/aorc_high/APCP_surface_2011*.nc', 'APCP_surface', 24, 126, 186)
data_aorc = data_aorc.reshape(-1, 24, 1, 126, 186).sum(axis=2)
mean_val = np.nanmean(data_aorc)
data_aorc = np.where(np.isnan(data_aorc), mean_val, data_aorc)
print(data_aorc.shape)

final shape (365, 24, 126, 186)
(365, 24, 126, 186)


In [ ]:
data_aorc_l = readFile('data/aorc_low/APCP_surface_2011*.nc', 'APCP_surface', 24, 21, 31)
data_aorc_l = data_aorc_l.reshape(-1, 6, 4, 21, 31).sum(axis=2)
mean_val = np.nanmean(data_aorc_l)
data_aorc_l = np.where(np.isnan(data_aorc_l), mean_val, data_aorc_l)
print(data_aorc_l.shape)

final shape (365, 24, 21, 31)
(365, 6, 21, 31)


In [5]:
input, target = make_temporal_batches(data_aorc_l, data_aorc)
print(input.shape)
print(target.shape)

(486, 1, 18, 21, 31)
(486, 1, 72, 126, 186)


In [6]:
free_mem, total_mem = torch.cuda.mem_get_info()
free_mem_MB = free_mem / 1024**2
total_mem_MB = total_mem / 1024**2

print(f"Remaining: {free_mem_MB:.2f} MB")
print(f"Total: {total_mem_MB:.2f} MB")

Remaining: 32191.12 MB
Total: 32501.12 MB


In [ ]:
train(input, target)